In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import Model
import tensorflow as tf

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [ ]:
train_dir = 'dataset/train_images'
val_dir = 'dataset/val_images'
test_dir = 'dataset/test_images'
train_dir2 = 'train'

#datagen = ImageDataGenerator(1./255)
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=32,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'training'
)
val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=32,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'validation'
)
print(train_generator.class_names)

In [ ]:
def inception_module(layer_in, f1,f2_in,f2_out,f3_in,f3_out,f4_out):
    #1x1
    conv1 = Conv2D(f1,(1,1),padding='same',activation='relu')(layer_in)
    
    #1x1 with 3x3
    tower_1 = Conv2D(f2_in,(1,1),padding='same',activation='relu')(layer_in)
    conv3 = Conv2D(f2_out,(3,3),padding='same',activation='relu')(tower_1)
    
    #1x1 with 5x5
    tower_2 = Conv2D(f3_in,(1,1),padding='same',activation='relu')(layer_in)
    conv5 = Conv2D(f3_out,(3,3),padding='same',activation='relu')(tower_2)
    conv5 = Conv2D(f3_out,(3,3),padding='same',activation='relu')(conv5)
    
    #3x3 max pooling with 
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    conv1_2 = Conv2D(f4_out, (1,1),padding='same',activation='relu')(tower_3)
    
    layer_out = tf.concat([conv1,conv3,conv5,conv1_2], axis=-1)
    
    return layer_out
    

In [ ]:
visible = Input(shape=(256,256,3))

x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(visible)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPooling2D(pool_size =2, strides =2, padding ='same')(x)

x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPooling2D(pool_size =2, strides =2, padding ='same')(x)


layer = inception_module(x,64,96,128,16,32,32)
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)

#x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(layer)
#x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
"""x = MaxPooling2D(pool_size =2, strides =2, padding ='same')(x)

x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPooling2D(pool_size =2, strides =2, padding ='same')(x)"""

"""x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPooling2D(pool_size =2, strides =2, padding ='same')(x)"""

#flat = Flatten()(x)
"""out = Dense(units=256,activation='relu')(flat)
out = Dense(units=128,activation='relu')(flat)
out = Dense(units=64,activation='relu')(flat)"""

out = GlobalAveragePooling2D()(layer)
out = Flatten()(out)
out = Dropout(0.2)(out)
out = Dense(2,activation='softmax')(out)
model = Model(inputs=visible,outputs=out)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=10)

In [ ]:
history2 = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=40)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image

img = image.load_img('dataset/test_images/fake/df2.jpg', target_size=(256,256))
img_1 = Image.open('dataset/test_images/fake/df2.jpg')
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

img2 = image.load_img('dataset/test_images/real/real00240.jpg', target_size=(256,256))
img_2 = Image.open('dataset/test_images/real/real00240.jpg')
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis = 0)

classes = ['fake','real']

#print(classes[np.argmax(model.predict(img))])
#print(classes[np.argmax(model.predict(img2))])

plt.imshow(img_1)
plt.title(classes[np.argmax(model.predict(img))])

In [ ]:
plt.imshow(img_2)
plt.title(classes[np.argmax(model.predict(img2))])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(40)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:

test_dir = 'test'

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb'
)

score = model.evaluate(test_generator)

print(score)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model/vgg16/vgg16_4.h5')
model.summary()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img('dataset/test_images/fake/df2.jpg',target_size=(256,256))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
img /= 255.

img.shape

In [ ]:
top4_layers = [x.output for x in model.layers[-6:]]
top4_model = Model(inputs=model.input,outputs=top4_layers)

pre = top4_model.predict(img)

In [ ]:
import matplotlib.pyplot as plt

f1 = pre[0]
print(f1.shape)
plt.matshow(f1[0,:,:,1],cmap='viridis')
plt.show()

In [ ]:
import tensorflow.keras as keras

# These are the names of the layers, so can have them as part of our plot
layer_names = []
for layer in model.layers[-6:]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, pre):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()